In [ ]:
# !pip install crewai
# !pip install 'crewai[tools]'

In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_ENDPOINT"] = ""

In [5]:
from crewai_tools import PDFSearchTool
tool = PDFSearchTool(
    config=dict(
        llm=dict(
            provider="openai", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gpt-4.5-turbo",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="openai", # or openai, ollama, ...
            config=dict(
                model="text-embeddings-v1",
                # task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

2024-05-23 09:24:06,719 - 139796167472960 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
2024-05-23 09:24:06,741 - 139796167472960 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
2024-05-23 09:24:06,744 - 139796167472960 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


In [9]:
from crewai import Agent
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

In [10]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

In [11]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

2024-05-23 09:07:48,547 - 140280652883776 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed
2024-05-23 09:07:48,553 - 140280652883776 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
2024-05-23 09:07:48,572 - 140280652883776 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
2024-05-23 09:07:48,575 - 140280652883776 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
2024-05-23 09:07:48,642 - 140280652883776 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
2024-05-23 09:07:48,650 - 140280652883776 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientStartEvent: module 'chromad

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
print(result)

In [2]:
from crewai_tools import BaseTool
from io import BytesIO
import pytesseract
from PIL import Image
import fitz
# pip install PyMuPDF

class PDFReaderTool(BaseTool):
    name: str = "PDFReaderTool"
    description: str = "Ferramenta para ler e extrair texto de arquivos PDF."

    def _run(self, file_path: str) -> str:
        return " ".join(self.read_pdf(file_path))
    
    @staticmethod
    def read_image(image_path: bytes = None, image_bytes: bytes = None) -> str:
        if image_path:
            image = Image.open(image_path)
        elif image_bytes:
            image = Image.open(BytesIO(image_bytes))
        else:
            raise ValueError("Either image_path or image_bytes must be provided.")
        
        return pytesseract.image_to_string(image)

    def read_pdf(self, pdf_path: bytes) -> str:
        # doc = fitz.open(stream=pdf_bytes, filetype="pdf")
        doc = fitz.open(pdf_path)
        texts = []
        for page in range(doc.page_count):
            page = doc[page]
            image_pix_map = page.get_pixmap()
            # Convert to tensor
            # image = tf.io.decode_jpeg(image_pix_map.tobytes())
            image_bytes = image_pix_map.tobytes()
            text = self.read_image(image_bytes=image_bytes)
            texts.append(text)
        return texts


2024-05-23 15:07:10,735 [embedchain] [INFO] Swapped std-lib sqlite3 with pysqlite3 for ChromaDb compatibility. Your original version was 3.31.1.


In [6]:
AZURE_GPT4_API_KEY=""
AZURE_GPT4_API_VERSION=""
AZURE_GPT4_ENDPOINT=""
AZURE_GPT4_CHAT_DEPLOYMENT_NAME=""


In [8]:
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv()
azure_llm = AzureChatOpenAI(
    api_key=AZURE_GPT4_API_KEY,
    openai_api_version=AZURE_GPT4_API_VERSION,
    azure_endpoint=AZURE_GPT4_ENDPOINT,
    azure_deployment=AZURE_GPT4_CHAT_DEPLOYMENT_NAME,
    temperature=0,
    request_timeout=60,
    max_retries=3,
)

In [24]:
from typing import List
from crewai import Agent, Task, Crew, Process
# from langchain.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field

class PersonData(BaseModel):
    name: str = Field(..., description="The person's full name")
    marital_status: str = Field(..., description="The person's marital status")
    nationality: str = Field(..., description="The person's nationality")
    tax_number: str = Field(..., description="The person's tax identification number")
    street_name: str = Field(..., description="The name of the street where the person lives")
    street_number: str = Field(..., description="The number of the building on the street where the person lives")
    unit_number: str = Field(..., description="The number of the unit where the person lives, if applicable")
    neighborhood: str = Field(..., description="The neighborhood where the person lives")
    city: str = Field(..., description="The city where the person lives")
    role: str = Field(..., description="The person's role within the organization")
    postalcode: str = Field(..., description="The postal code of the person's address")
    state: str = Field(..., description="The state where the person lives")
    group: str = Field(..., description="The group to which the person belongs within the organization")
    contributor_status: str = Field(..., description="The person's status as a contributor to the organization")
    person_type: str = Field(..., description="The type of person, such as employee, contractor, etc.")
    representativeness: str = Field(..., description="The person's level of representativeness within the organization")
    representation: str = Field(..., description="The type of representation the person has within the organization")
    participation_percentage: str = Field(..., description="The percentage of participation the person has within the organization")
    quotas_number: str = Field(..., description="The number of quotas the person has within the organization")
    quotas_value: str = Field(..., description="The value of the quotas the person has within the organization")
    nire: str = Field(..., description="The NIRE (Registro de Comércio) number of the person")
    social_capital: str = Field(..., description="The amount of social capital the person has within the organization")
    establishment: str = Field(..., description="The establishment where the person works")


class FirmasData(BaseModel):
    administrators: List[PersonData] = Field(..., description="List of administrators")
    prosecutors: List[PersonData] = Field(..., description="List of prosecutors")
    companies: List[PersonData] = Field(..., description="List of companies")
    unclassified: List[PersonData] = Field(..., description="List of unclassified persons")
        
        
class DocumentData(BaseModel):
    document_type: str = Field(..., description="Type of the document")
    signature_date: str = Field(..., description="Date of the signature")
    document_expiration: str = Field(..., description="Expiration date of the document")
    issue_date: str = Field(..., description="Issue date of the document")
    expiration: str = Field(..., description="Expiration date")
    procuration_type: str = Field(..., description="Type of procuration")
    procuration_subtype: str = Field(..., description="Subtype of procuration")
    document_record: str = Field(..., description="Record of the document")
    document_record_date: str = Field(..., description="Date of the document record")
    act: str = Field(..., description="Act of the document")
    alteration_number: str = Field(..., description="Number of alterations")
    meeting_date: str = Field(..., description="Date of the meeting")
    changes_flags: str = Field(..., description="Flags for changes")
    document_title: str = Field(..., description="Title of the document")
        
        
class Summary(BaseModel):
    firmas_data: FirmasData
    document_data: DocumentData


In [ ]:
from crewai import Agent, Task, Crew, Process

# Crie a ferramenta PDF Reader
pdf_reader_tool = PDFReaderTool()

# Crie um agente que utilizará a ferramenta PDF Reader
leitor_pdf = Agent(
    role='Leitor de PDF',
    goal='Extrair e analisar informações de documentos PDF',
    verbose=False,
    memory=True,
    backstory=(
        "Você é um especialista em leitura e análise de documentos, "
        "capaz de extrair informações importantes de arquivos PDF."
    ),
    tools=[pdf_reader_tool],
    allow_delegation=True,
    llm=azure_llm
)

# Crie uma tarefa para o agente leitor de PDF
tarefa_extracao = Task(
    description=(
        "Leia o arquivo PDF no path {file_path} e aprenda sobre o conteúdo."
    ),
    expected_output='Retorne um objeto Pydantic com as informações extraídas do PDF.',
    output_pydantic=Summary,
    tools=[pdf_reader_tool],
    agent=leitor_pdf
)
# Formar a equipe com o agente configurado
equipe = Crew(
    agents=[leitor_pdf],
    tasks=[tarefa_extracao],
    process=Process.sequential  # Execução sequencial das tarefas é o padrão
)

# Iniciar a equipe e extrair informações do PDF fornecendo o caminho correto
resultado = equipe.kickoff(
    inputs={
        'file_path': 'docs/65f08f665cf6e92fbd3e0293.pdf',
    })
print(resultado)


In [31]:
# BaseModel to dict
summary_dict = resultado.dict()
summary_dict

{'firmas_data': {'administrators': [{'name': 'Ana Fava Auriemo',
    'marital_status': 'divorciada',
    'nationality': 'brasileira',
    'tax_number': '275.225.148-35',
    'street_name': 'Avenida Magalhiies de Castro',
    'street_number': '4800',
    'unit_number': '21',
    'neighborhood': '',
    'city': 'São Paulo',
    'role': 'Administradora',
    'postalcode': '05502-001',
    'state': 'SP',
    'group': '',
    'contributor_status': '',
    'person_type': 'physical',
    'representativeness': '',
    'representation': '',
    'participation_percentage': '',
    'quotas_number': '5.153.234',
    'quotas_value': '5.153.234,00',
    'nire': '',
    'social_capital': '42324.626,00',
    'establishment': ''}],
  'prosecutors': [],
  'companies': [{'name': 'TERRA ALTA IMÓVEIS LTDA.',
    'marital_status': '',
    'nationality': '',
    'tax_number': '10.848.165/0001-71',
    'street_name': 'Avenida Magalhiies de Castro',
    'street_number': '4800',
    'unit_number': '14',
    'ne